In [1]:
import argparse
import gc
import logging
import os
import glob
import pandas as pd

import sys
# sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')


import time

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from math import exp

import numpy as np
torch.backends.cudnn.benchmark = True

from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

num = 10

/home/vikrant/core/home/R/goal/baselines/deep_prediction/argoverse-api/argoverse/map_representation/map_api.py:75: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  lane_polygons = np.array(self.get_vector_map_lane_polygons(city_name))
/home/vikrant/core/home/R/goal/baselines/deep_prediction/argoverse-api/argoverse/map_representation/map_api.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  driveable_areas = np.array(self.get_vector_map_driveable_areas(city_name))


In [ ]:
path = "/datasets/argoverse/forecasting_sample/data/"
dfs = [pd.read_csv(file) for file in  glob.glob(os.path.join(path, "*.csv"))]

In [ ]:
df = dfs[0]
agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
others_df = df[df['OBJECT_TYPE'] == 'OTHERS']
others_dfs = np.array([v for k, v in others_df.groupby('TRACK_ID')], dtype=object)
av_df = df[df['OBJECT_TYPE'] == 'AV']
agent_df['X'].values
df

In [ ]:
import os
import sys
sys.path.append("../ddn/")
sys.path.append("./")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
#from bernstein import bernstesin_coeff_order10_new

from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

def denoise(gt_x, gt_y, w = 7):
    # denoising
    gt_x_t = []
    gt_y_t = []
    for iq in range(len(gt_x)):
        if iq >= w and iq + w <= len(gt_x):
            gt_x_t.append(np.mean(gt_x[iq: iq + w]))
            gt_y_t.append(np.mean(gt_y[iq: iq + w]))
        elif iq < w:
            okx = np.mean(gt_x[w: w + w])
            gt_x_t.append(gt_x[0] + (okx - gt_x[0]) * (iq) / w)
            oky = np.mean(gt_y[w: w + w])
            gt_y_t.append(gt_y[0] + (oky - gt_y[0]) * (iq) / w)
        else:
            okx = np.mean(gt_x[len(gt_x) - w:len(gt_x) - w  + w])
            oky = np.mean(gt_y[len(gt_x) - w: len(gt_x) - w + w])
            gt_x_t.append(okx + (gt_x[-1] - okx) * (w - (len(gt_x) - iq)) / w)
            gt_y_t.append(oky + (gt_y[-1] - oky) * (w - (len(gt_y) - iq)) / w)                   

    gt_x = gt_x_t
    gt_y = gt_y_t
    return gt_x, gt_y

def rotate(gt_x, gt_y,theta):
    gt_x_x = [ (gt_x[k] * np.cos(theta) - gt_y[k] * np.sin(theta))  for k in range(len(gt_x))]
    gt_y_y = [ (gt_x[k] * np.sin(theta) + gt_y[k] * np.cos(theta))  for k in range(len(gt_x))]
    gt_x = gt_x_x
    gt_y = gt_y_y
    return gt_x, gt_y

def transform(x_traj, y_traj, dt = 0.3, t_obs = 20, theta=None, offsets=None):
    if offsets == None:
        x_traj -= x_traj[0]
        y_traj -= y_traj[0]
    else:
        x_traj -= offsets[0]
        y_traj -= offsets[1]
        
    gt_x = x_traj
    gt_y = y_traj
    
    gt_x, gt_y = denoise(gt_x, gt_y)
    v_x = [ (gt_x[k + 1] - gt_x[k])/dt  for k in range(len(gt_x) - 1)]
    v_y = [ (gt_y[k + 1] - gt_y[k])/dt  for k in range(len(gt_y) - 1)]
    psi = [ np.arctan2(v_y[k], v_x[k]) for k in range(len(v_x))]  

    # till here, gt-> (50, 1), v -> (49, 1), psi -> (31, 1)

    # obtain this -psi
    if theta == None:
        theta = -psi[t_obs - 1]

    # rotate by theta
    gt_x, gt_y = rotate(gt_x, gt_y, theta)
    v_x = [ (gt_x[k + 1] - gt_x[k])/dt  for k in range(len(gt_x) - 1)]
    v_y = [ (gt_y[k + 1] - gt_y[k])/dt  for k in range(len(gt_y) - 1)]
    psi = [ np.arctan2(v_y[k], v_x[k]) for k in range(len(v_x))]
    psidot = [ (psi[k + 1] - psi[k])/dt for k in range(len(psi) - 1) ]
    psi_traj = [i.item() for i in psi]
    psidot_traj = [i.item() for i in psidot]
    
    return gt_x, gt_y, v_x, v_y, psi, psidot, psi_traj, psidot_traj, theta

class ArgoverseDataset(Dataset):
    def __init__(self, data_path, t_obs=16, dt=0.125,centerline_dir=None, include_centerline = False, flatten=True):
        self.paths = glob.glob(os.path.join(path, "*.csv"))
        self.data_path = data_path
        self.t_obs = t_obs
        self.dt = dt
        self.include_centerline = include_centerline
        self.centerline_dir = centerline_dir
        self.flatten = flatten
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        dt = self.dt
        path = self.paths[idx]
        df = pd.read_csv(path)
        agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
        others_df = df[df['OBJECT_TYPE'] == 'OTHERS']
        others_dfs = np.array([v for k, v in others_df.groupby('TRACK_ID')], dtype=object)
        av_df = df[df['OBJECT_TYPE'] == 'AV']        

        x_traj = agent_df['X'].values
        y_traj = agent_df['Y'].values                

        # transformation
        x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta = transform(x_traj, y_traj, dt = self.dt, t_obs = self.t_obs)                

        x_inp = x_traj[:self.t_obs]
        y_inp = y_traj[:self.t_obs]
        x_fut = x_traj[self.t_obs:]
        y_fut = y_traj[self.t_obs:]

        psi_fut = psi_traj[self.t_obs - 1:]
        psidot_fut = psi_traj[self.t_obs - 2:]
        
        vx_traj = v_x
        vy_traj = v_y
        
        vx_beg = vx_traj[self.t_obs]
        vy_beg = vy_traj[self.t_obs]
        
        vx_beg_prev = vx_traj[self.t_obs - 1]
        vy_beg_prev = vy_traj[self.t_obs - 1]
        
        ax_beg = (vx_beg - vx_beg_prev) / self.dt
        ay_beg = (vy_beg - vy_beg_prev) / self.dt

        vx_fin = v_x[-1]
        vy_fin = v_y[-1]
        
        vx_fin_prev = v_x[-2]
        vy_fin_prev = v_y[-2]

        ax_fin = (vx_fin - vx_fin_prev) / self.dt
        ay_fin = (vy_fin - vy_fin_prev) / self.dt

        x_fut = x_traj[self.t_obs:]
        y_fut = y_traj[self.t_obs:]
        
        if self.flatten:
            traj_inp = np.dstack((x_inp, y_inp)).flatten()  
        else:
            traj_inp = np.vstack((x_inp, y_inp))
            traj_inp = np.swapaxes(traj_inp, 0, 1)

        vx_fut = vx_traj[self.t_obs:]
        vy_fut = vy_traj[self.t_obs:]
        traj_out = np.hstack((x_fut, y_fut)).flatten()

        fixed_params = np.array([x_fut[0], y_fut[0], 0, psi_fut[0], psidot_fut[0]])
        var_inp = np.array([x_inp[-1], y_inp[-1], psi_fut[-1]])

        return torch.tensor(traj_inp), torch.tensor(traj_out), torch.tensor(fixed_params), torch.tensor(var_inp)

In [ ]:
predictions = np.load('/home/vikrant/core/home/R/goal/code/prediction.npy', allow_pickle=True)
gt = np.load('/datasets/argoverse/val_data.npy', allow_pickle=True)

In [ ]:
train_dataset = ArgoverseDataset("/datasets/argoverse/forecasting_sample/data", t_obs=20, dt=0.3)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=False, num_workers=0)

In [ ]:
data_path="/datasets/argoverse/val/data"
output_dir="../results/"
t_obs=20
dt=0.3
t_obs=20
pred=False
pred_array=None
batch_size = 512
dpi=50
w,h=200,200
paths = glob.glob(os.path.join(data_path, "*.csv"))
color = {
    'polygon': '#e6cf93',
    'polygon-outline': '#e6cf93',
    'centerline': '#fceec7',
    'agent': 'blue',
    'av': 'grey',
    'other': 'grey',
    'outline': 'black'
}
from tqdm import tqdm
for idx in tqdm(range(len(paths))):
    path = paths[idx]
    dff = pd.read_csv(path)
    
    city = dff['CITY_NAME'].values[0]    
    
    agent_df = dff[dff['OBJECT_TYPE'] == 'AGENT']
    x_a = agent_df['X'].values
    y_a = agent_df['Y'].values    
    x_a, y_a = denoise(x_a, y_a)    
    
    av_df = dff[dff['OBJECT_TYPE'] == 'AV']
    x_av = av_df['X'].values
    y_av = av_df['Y'].values
    x_av, y_av = denoise(x_av, y_av)
    
    others_df = dff[dff['OBJECT_TYPE'] == 'OTHERS']
    others_dfs = np.array([v for k, v in others_df.groupby('TRACK_ID')], dtype=object)
    x_o = {}
    y_o = {}
    
    for other_df in others_dfs:
        x_other, y_other = other_df['X'].values, other_df['Y'].values
        x_other, y_other = denoise(x_other, y_other)
        x_o[other_df['TRACK_ID'].values[0]] = x_other
        y_o[other_df['TRACK_ID'].values[0]] = other_df['Y'].values      
    
    # group by timestamp
    dfs = [x for _, x in dff.groupby('TIMESTAMP')]    
    
    for ind, df in enumerate(dfs):
        agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
        others_df = df[df['OBJECT_TYPE'] == 'OTHERS']
        others_dfs = [x for _, x in others_df.groupby('TRACK_ID')]
#         others_dfs = np.array([v for k, v in others_df.groupby('TRACK_ID')], dtype=object)
        av_df = df[df['OBJECT_TYPE'] == 'AV']

        # agent
        x_traj = agent_df['X'].values
        y_traj = agent_df['Y'].values
        offsets = [x_a[0], y_a[0]] # offsets for other agents
    #         x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta_agent = transform(x_traj, y_traj)
        fig = plt.figure(figsize=(200/dpi,200/dpi), dpi=dpi)
# #         fig = plt.figure(figsize=(10, 10), dpi=dpi)

        x_off = 75
        y_off = 75
        points = np.array([[x_a[20] - x_off, y_a[20] + y_off],[x_a[20] + x_off, y_a[20] + y_off], [x_a[20] + x_off, y_a[20] - y_off],[x_a[20] - x_off, y_a[20] - y_off],[x_a[20] - x_off, y_a[20] + y_off]])
        plt.fill(points[:, 0], points[:, 1], color=color['outline'], zorder=0)
# #             ax = fig.add_subplot(1, 1, 1) # nrows, ncols, index
# # #         plt.plot(x_traj[:t_obs], y_traj[:t_obs], color='blue', label='observed')
        if ind < len(dfs) - 1:
            x_off = 0.75
            y_off = 1.25
            points = np.array([[x_traj[0] - x_off, y_traj + y_off],[x_traj[0] + x_off, y_traj + y_off], [x_traj[0] + x_off, y_traj - y_off],[x_traj[0] - x_off, y_traj - y_off],[x_traj[0] - x_off, y_traj + y_off]])
#             plt.fill(points[:, 0], points[:, 1], color='blue', zorder=5)
            theta = np.arctan2((y_a[ind + 1] - y_a[ind]) , (x_a[ind + 1] - x_a[ind])) - np.pi/2
#             print(theta)
            w = np.zeros(points.shape)
            A = np.matrix([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
            points = points - np.array([x_traj[0], y_traj[0]])
            for i,v in enumerate(points): w[i] = A @ points[i]
            plt.fill(w[:, 0] + x_traj[0], w[:, 1] + y_traj[0], color=color['agent'], zorder=5)
        plt.scatter(x_traj[0], y_traj[0], color=color['agent'], label='end observed', zorder=5)

# #         plt.plot(x_traj[t_obs:], y_traj[t_obs:], color='orange', label='gt')
#         plt.scatter(x_traj[-1], y_traj[-1], color='orange', label='gt end point')

        # av
        x_traj = av_df['X'].values
        y_traj = av_df['Y'].values
        x_max, y_max = np.max(x_traj), np.max(y_traj)
        x_min, y_min = np.min(x_traj), np.min(y_traj)

        if ind < len(dfs) - 1:
            x_off = 0.75
            y_off = 1.25
            points = np.array([[x_traj[0] - x_off, y_traj + y_off],[x_traj[0] + x_off, y_traj + y_off], [x_traj[0] + x_off, y_traj - y_off],[x_traj[0] - x_off, y_traj - y_off],[x_traj[0] - x_off, y_traj + y_off]])
#             plt.fill(points[:, 0], points[:, 1], color='blue', zorder=5)
            theta = np.arctan2((y_av[ind + 1] - y_av[ind]) , (x_av[ind + 1] - x_av[ind])) - np.pi/2
#             print(theta)
            w = np.zeros(points.shape)
            A = np.matrix([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
            points = points - np.array([x_traj[0], y_traj[0]])
            for i,v in enumerate(points): w[i] = A @ points[i]
            plt.fill(w[:, 0] + x_traj[0], w[:, 1] + y_traj[0], color=color['av'], zorder=4)
        
#         plt.plot(x_traj, y_traj, color='red', label='AV')
        plt.scatter(x_traj[-1], y_traj[-1], color=color['av'], zorder=4)

#         # others
        for indoo, other in enumerate(others_dfs):
            x_traj = other['X'].values
            y_traj = other['Y'].values
            indo = other['TRACK_ID'].values[0]
            if ind < len(dfs) - 1 and ind < len(x_o[indo]) - 1 and ind < len(y_o[indo]) - 1:
                x_off = 0.75
                y_off = 1.25
                points = np.array([[x_traj[0] - x_off, y_traj + y_off],[x_traj[0] + x_off, y_traj + y_off], [x_traj[0] + x_off, y_traj - y_off],[x_traj[0] - x_off, y_traj - y_off],[x_traj[0] - x_off, y_traj + y_off]])
    #             plt.fill(points[:, 0], points[:, 1], color='blue', zorder=5)
                
                theta = np.arctan2((y_o[indo][ind + 1] - y_o[indo][ind]) , (x_o[indo][ind + 1] - x_o[indo][ind])) - np.pi/2
    #             print(theta)
                w = np.zeros(points.shape)
                A = np.matrix([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
                points = points - np.array([x_traj[0], y_traj[0]])
                for i,v in enumerate(points): w[i] = A @ points[i]
                plt.fill(w[:, 0] + x_traj[0], w[:, 1] + y_traj[0], color=color['other'], zorder=4)
            
#             x_max, y_max = np.max(x_traj), np.max(y_traj)
#             x_min, y_min = np.min(x_traj), np.min(y_traj)

# #     #             x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta_others = transform(x_traj, y_traj, theta = theta_agent, offsets = offsets)
#             plt.plot(x_traj, y_traj, color='grey')
#             plt.scatter(x_traj[-1], y_traj[-1], color='grey')

#         # centerlines
        lane_centerlines = []    
        # Get lane centerlines which lie within the range of trajectories
        agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
        gt_x = agent_df['X'].values
        gt_y = agent_df['Y'].values

        x_max, y_max = np.max(x_a) + 50, np.max(y_a) + 50
        x_min, y_min = np.min(x_a) - 50, np.min(y_a) - 50

        print(x_max, x_min)
        print(y_max, y_min)
        avm = ArgoverseMap()
        for arr in avm.find_local_lane_polygons([x_min, x_max, y_min, y_max], city):
            plt.fill(arr[:, 0], arr[:, 1], color=color['polygon'],zorder=0)

        for arr in avm.find_local_lane_polygons([x_min, x_max, y_min, y_max], city):
            plt.plot(arr[:, 0], arr[:, 1], color=color['polygon-outline'],zorder=1)

        seq_lane_props = avm.city_lane_centerlines_dict[city]
        for lane_id, lane_props in seq_lane_props.items():
            lane_cl = lane_props.centerline
            if (np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min):
                lane_centerlines.append(lane_cl)

        for lane_cl in lane_centerlines:
            plt.plot(lane_cl[:, 0], lane_cl[:, 1], color=color['centerline'], alpha=1, linewidth=1, zorder=2)

#         plt.legend()
        plt.xlim([x_a[20] - 50, x_a[20] + 50])
        plt.ylim([y_a[20] - 50, y_a[20] + 50])
        import os
        try:
            os.mkdir('../results/{}'.format(idx))
        except:
            pass
#         plt.set_facecolor('red')
        plt.axis('off')
#         fig.canvas.draw()
#         data_image = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
#         data_image = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
#         print(data_image.shape)

        # Probability as a function of distance from the center derived
        # from a gaussian distribution with mean = 0 and stdv = 1
        scaledGaussian = lambda x : exp(-(1/2)*(x**2))

        # Convert Grayscale to HeatMap Using Opencv
        plt.savefig('../results/{}/{}.png'.format(idx,ind), dpi=dpi, bbox_inches='tight')
#         plt.imshow(isotropicGrayscaleImage, cmap='hot', interpolation='nearest', zorder=7, alpha=0.7)
#         plt.show()
        plt.clf()
        break
    np.save("../results/offsets/{}.npy".format(idx), np.array([x_a[49], y_a[49]]))
#     break

In [ ]:
import cv2
from PIL import Image
path = '../results/0/0.png'
img = cv2.imread(path)
image = Image.open('../results/0/0.png')
print(np.asarray(image).shape)
print(img.shape)

In [ ]:
# plt.figure(figsize=(10, 10))
# plt.imshow(data, interpolation='nearest')

In [ ]:
def vis_trajectories(data_path, output_dir="results/", dt = 0.3, t_obs=20, pred=False, pred_array=None, batch_size = 512):
    paths = glob.glob(os.path.join(data_path, "*.csv"))
    for idx in range(len(paths)):
        path = paths[idx]
        df = pd.read_csv(path)
        agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
        others_df = df[df['OBJECT_TYPE'] == 'OTHERS']
        others_dfs = np.array([v for k, v in others_df.groupby('TRACK_ID')], dtype=object)
        av_df = df[df['OBJECT_TYPE'] == 'AV']
        city = df['CITY_NAME'].values[0]

        # agent
        x_traj = agent_df['X'].values
        y_traj = agent_df['Y'].values
        x_a, y_a = x_traj, y_traj
        x_max, y_max = np.max(x_traj), np.max(y_traj)
        x_min, y_min = np.min(x_traj), np.min(y_traj)

        offsets = [x_traj[0], y_traj[0]] # offsets for other agents
    #         x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta_agent = transform(x_traj, y_traj)
        plt.figure(figsize=(15,15))

        plt.plot(x_traj[:t_obs], y_traj[:t_obs], color='blue', label='observed')
        plt.scatter(x_traj[t_obs], y_traj[t_obs], color='blue', label='end observed')
        plt.plot(x_traj[t_obs:], y_traj[t_obs:], color='orange', label='gt')
        plt.scatter(x_traj[-1], y_traj[-1], color='orange', label='gt end point')
        if pred:
            x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta_agent = transform(x_traj, y_traj)
            pred_x = pred_array[idx][:, 0]
            pred_y = pred_array[idx][:, 1]
            pred_x, pred_y = rotate(pred_x, pred_y, -theta)
            pred_x += offsets[0]
            pred_y += offsets[1]
            plt.plot(pred_x, pred_y, color='orange', label='predicted')
            plt.scatter(pred_x[-1], pred_y[-1], color='orange', label='predicted goal point')    

        # av
        x_traj = av_df['X'].values
        y_traj = av_df['Y'].values
        x_max, y_max = np.max(x_traj), np.max(y_traj)
        x_min, y_min = np.min(x_traj), np.min(y_traj)

        plt.plot(x_traj, y_traj, color='red', label='AV')
        plt.scatter(x_traj[-1], y_traj[-1], color='red')

        # others
        for other in others_dfs:
            x_traj = other['X'].values
            y_traj = other['Y'].values
            x_max, y_max = np.max(x_traj), np.max(y_traj)
            x_min, y_min = np.min(x_traj), np.min(y_traj)

    #             x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta_others = transform(x_traj, y_traj, theta = theta_agent, offsets = offsets)
            plt.plot(x_traj, y_traj, color='grey')
            plt.scatter(x_traj[-1], y_traj[-1], color='grey')

        # centerlines
        lane_centerlines = []    
        # Get lane centerlines which lie within the range of trajectories
        agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
        gt_x = agent_df['X'].values
        gt_y = agent_df['Y'].values

        x_max, y_max = np.max(gt_x) + 30, np.max(gt_y) + 30
        x_min, y_min = np.min(gt_x) - 30, np.min(gt_y) - 30

        print(x_max, x_min)
        print(y_max, y_min)
        avm = ArgoverseMap()
        for arr in avm.find_local_lane_polygons([x_min, x_max, y_min, y_max], city):
            plt.plot(arr[:, 0], arr[:, 1], color='black')

        seq_lane_props = avm.city_lane_centerlines_dict[city]
        for lane_id, lane_props in seq_lane_props.items():
            lane_cl = lane_props.centerline
            if (np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min):
                lane_centerlines.append(lane_cl)

        for lane_cl in lane_centerlines:
            plt.plot(lane_cl[:, 0], lane_cl[:, 1], "--", color="grey", alpha=1, linewidth=1, zorder=0)

        plt.legend()
        plt.xlim([x_a[20] - 50, x_a[20] + 50])
        plt.ylim([y_a[20] - 50, y_a[20] + 50])
        plt.show()
        break

In [ ]:
# # predictions = np.load('../prediction.npy', allow_pickle=True)
vis_trajectories(data_path = "/datasets/argoverse/val/data", output_dir="../results/", t_obs=20)

In [ ]:
import pandas as pd
data_ = pd.read_pickle('/datasets/argoverse/forecasting_features_val.pkl')
features = data_["FEATURES"].values
features[0].shape

In [ ]:
for i in range(num):
    city = features[i][0][5]
    seq_lane_props = avm.city_lane_centerlines_dict[city]
    
    gt_x = features[i][:, 3]
    gt_y = features[i][:, 4]
    
    x_max, y_max = np.max(gt_x) + 50, np.max(gt_y) + 50
    x_min, y_min = np.min(gt_x) - 50, np.min(gt_y) - 50
    print(x_max, x_min)
    print(y_max, y_min)
    
    lane_centerlines = []    
    # Get lane centerlines which lie within the range of trajectories
    for lane_id, lane_props in seq_lane_props.items():
        lane_cl = lane_props.centerline

        if (np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min):
            lane_centerlines.append(lane_cl)

    for lane_cl in lane_centerlines:
        plt.plot(lane_cl[:, 0], lane_cl[:, 1], "--", color="grey", alpha=1, linewidth=1, zorder=0)
    
    plt.plot(gt_x[:20], gt_y[:20], color='blue', label='observed')
    plt.plot(gt_x[20:], gt_y[20:], color='orange', label='gt')
#     plt.plot(pred_x[20:], pred_y[20:], color='red', label='predicted')
    plt.legend()
    plt.show()
    

In [ ]:
for i in range(num):
    city = features[i][0][5]
    seq_lane_props = avm.city_lane_centerlines_dict[city]
    
    gt_x = features[i][:, 3]
    gt_y = features[i][:, 4]
    
    x_max, y_max = np.max(gt_x) + 50, np.max(gt_y) + 50
    x_min, y_min = np.min(gt_x) - 50, np.min(gt_y) - 50
    print(x_max, x_min)
    print(y_max, y_min)
    
    lane_centerlines = []
    plt.figure(figsize=(10, 10))
    # Get lane centerlines which lie within the range of trajectories
    for arr in avm.find_local_lane_polygons([x_min, x_max, y_min, y_max], city):
        plt.plot(arr[:, 0], arr[:, 1], color='black')
        plt.xlim([x_min, x_max])
        plt.ylim([y_min, y_max])
    for lane_id, lane_props in seq_lane_props.items():
        lane_cl = lane_props.centerline

        if (np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min):
            lane_centerlines.append(lane_cl)
    for lane_cl in lane_centerlines:
        plt.plot(lane_cl[:, 0], lane_cl[:, 1], "--", color="grey", alpha=1, linewidth=1, zorder=0)
    plt.plot(gt_x[:20], gt_y[:20], color='blue', label='observed')
    plt.plot(gt_x[20:], gt_y[20:], color='orange', label='gt')
#     plt.plot(pred_x[20:], pred_y[20:], color='red', label='predicted')
    plt.legend()
    plt.show()
    plt.clf()